In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
from unsloth import FastLanguageModel
import torch

# fourbit_models = [
#     "unsloth/Qwen3-1.7B-unsloth-bnb-4bit", # Qwen 14B 2x faster
#     "unsloth/Qwen3-4B-unsloth-bnb-4bit",
#     "unsloth/Qwen3-8B-unsloth-bnb-4bit",
#     "unsloth/Qwen3-14B-unsloth-bnb-4bit",
#     "unsloth/Qwen3-32B-unsloth-bnb-4bit",

#     # 4bit dynamic quants for superior accuracy and low memory use
#     "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
#     "unsloth/Phi-4",
#     "unsloth/Llama-3.1-8B",
#     "unsloth/Llama-3.2-3B",
#     "unsloth/Qwen3-4B",
#     "unsloth/Qwen3-8B"
#     "unsloth/Qwen3-14B"
#     "unsloth/orpheus-3b-0.1-ft-unsloth-bnb-4bit" # [NEW] We support TTS models!
# ] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B",
    max_seq_length = 2048,   # Context length - can be longer, but uses more memory
    load_in_4bit = False,     # 4bit uses much less memory
    load_in_8bit = True,    # A bit more accurate, uses 2x memory
    full_finetuning = True, # We have full finetuning now!
    # token = "hf_...",      # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/hoyeol/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-08 14:14:28.646062: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-08 14:14:28.659966: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749392068.675592   47115 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749392068.680378   47115 cuda_blas.cc:1407] Unable

🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
# model = FastLanguageModel.get_peft_model(
#     model,
#     r = 32,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
#     target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
#                       "gate_proj", "up_proj", "down_proj",],
#     lora_alpha = 32,  # Best to choose alpha = rank or rank*2
#     lora_dropout = 0, # Supports any, but = 0 is optimized
#     bias = "none",    # Supports any, but = "none" is optimized
#     # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
#     use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
#     random_state = 3407,
#     use_rslora = False,   # We support rank stabilized LoRA
#     loftq_config = None,  # And LoftQ
# )

In [4]:
import pandas as pd
train_caption = pd.read_csv("/home/hoyeol/projects/GCT634_final/data/DX7_YAMAHA_captions_train.csv", index_col=0)
valid_caption = pd.read_csv("/home/hoyeol/projects/GCT634_final/data/DX7_YAMAHA_captions_test.csv", index_col=0)
train_path = pd.read_csv("/home/hoyeol/projects/GCT634_final/data/DX7_YAMAHA_train_filtered.csv", index_col=0)
valid_path = pd.read_csv("/home/hoyeol/projects/GCT634_final/data/DX7_YAMAHA_test_filtered.csv", index_col=0)
train_df = pd.merge(train_path, train_caption[['id', 'caption']], on='id', how='left')
valid_df = pd.merge(valid_path, valid_caption[['id', 'caption']], on='id', how='left')

In [5]:
from datasets import load_dataset
from datasets import Dataset, DatasetDict
train_dataset = Dataset.from_pandas(train_df[['id', 'caption', 'patch_data']])
valid_dataset = Dataset.from_pandas(valid_df[['id', 'caption', 'patch_data']])

In [6]:
from prompt import zeroshot_prompt
import ast
from utils import serialize_specs
def proprocess(examples, prompt = zeroshot_prompt):
    conversations = []
    for caption, patch_data in zip(examples["caption"], examples["patch_data"]):
        question = prompt.format(prompt=caption)
        specs = ast.literal_eval(patch_data)
        specs.pop('name', None)
        specs.pop('has_fixed_freqs', None)
        solution = '```python\n' + 'specs = ' +serialize_specs(specs) + '```\n'
        conversations.append([
            {"role" : "user",      "content" : question},
            {"role" : "assistant", "content" : solution},
        ])
    return { "conversations": conversations, }

In [7]:
train_dataset = tokenizer.apply_chat_template(
    train_dataset.map(proprocess, batched = True)["conversations"],
    tokenize = False,
)
valid_dataset = tokenizer.apply_chat_template(
    valid_dataset.map(proprocess, batched = True)["conversations"],
    tokenize = False,
)
train_dataset = pd.Series(train_dataset)
valid_dataset = pd.Series(valid_dataset)
train_dataset.name = "text"
valid_dataset.name = "text"
train_dataset = Dataset.from_pandas(pd.DataFrame(train_dataset)).shuffle()
valid_dataset = Dataset.from_pandas(pd.DataFrame(valid_dataset))

Map: 100%|██████████| 63/63 [00:00<00:00, 4134.52 examples/s]


In [8]:
print(train_dataset[12]['text'])

<|im_start|>user
Find the parameter values for the famous 6-OP(operator) FM synthesizer, the DX7, such that the resulting sound matches the given prompt.
### Prompt: Koto, which has a bright, plucked timbre with a resonant decay.<|im_end|>
<|im_start|>assistant
<think>

</think>

```python
specs = {
    'modmatrix': [
		[0, 1, 0, 0, 0, 0],
		[0, 0, 0, 0, 0, 0],
		[0, 0, 0, 1, 1, 0],
		[0, 0, 0, 0, 0, 0],
		[0, 0, 0, 0, 0, 1],
		[0, 0, 0, 0, 0, 1]
    ],
    'outmatrix': [1, 0, 1, 0, 0, 0],
    'feedback': 7,
    'coarse': [3, 4, 1, 1, 4, 1],
    'fine': [0, 0, 0, 0, 0, 0],
    'detune': [-7, -7, -7, -7, -7, -7],
    'transpose': 0,
    'ol': [77, 83, 82, 99, 99, 94],
    'eg_rate': [
		[65, 83, 81, 60, 83, 89],
		[70, 37, 28, 64, 68, 62],
		[0, 29, 8, 28, 28, 58],
		[0, 15, 27, 32, 48, 34]
    ],
    'eg_level': [
		[99, 99, 99, 99, 99, 99],
		[88, 90, 76, 92, 83, 92],
		[0, 0, 0, 0, 0, 0],
		[0, 0, 0, 0, 0, 0]
    ],
    'sensitivity': [1, 1, 1, 5, 2, 4],
}```
<|im_end|>



In [9]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = valid_dataset, # Can set up evaluation!
    args = SFTConfig(
        eval_strategy="steps",
        eval_steps = 10,
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 10,
        num_train_epochs = 10, # Set this for 1 full training run.
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

Unsloth: Tokenizing ["text"] (num_proc=64): 100%|██████████| 600/600 [00:06<00:00, 96.59 examples/s] 
num_proc must be <= 63. Reducing num_proc to 63 for dataset of size 63.
Map (num_proc=64): 100%|██████████| 600/600 [00:00<00:00, 1096.41 examples/s]
num_proc must be <= 63. Reducing num_proc to 63 for dataset of size 63.
Map (num_proc=63): 100%|██████████| 63/63 [00:00<00:00, 102.88 examples/s]


In [10]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 600 | Num Epochs = 10 | Total steps = 750
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 1,720,574,976/1,720,574,976 (100.00% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
10,0.655300,0.619199
20,0.372600,0.370808
30,0.350000,0.350641
40,0.310300,0.343256
50,0.332700,0.338620
60,0.333000,0.335352
70,0.314800,0.332908
80,0.300000,0.330187
90,0.342000,0.330670
100,0.327800,0.327052


Unsloth: Not an error, but Qwen3Model does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


In [11]:
#0.345100	0.356628

In [ ]:
caption = train_df.iloc[5]['caption']
messages = [
    {"role" : "user", "content" : zeroshot_prompt.format(prompt=caption)}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = False, # Disable thinking
)
print(caption)

from transformers import TextStreamer
output = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 1024, # Increase for longer outputs!
    temperature = 1.0, #, top_p = 0.8, top_k = 20, # For non thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Electronic pad, which has a lush and evolving timbre with a slow attack and long release, creating a sustained and atmospheric texture.
```python
specs = {
    'modmatrix': [
		[0, 1, 0, 0, 0, 0],
		[0, 1, 0, 0, 0, 0],
		[0, 0, 0, 1, 0, 0],
		[0, 0, 0, 0, 0, 0],
		[0, 0, 0, 0, 0, 1],
    'outmatrix': [1, 0, 0, 0, 1, 0],
    'feedback': 5,
    'coarse': [1, 1, 1, 1, 1, 1],
    'fine': [0, 0, 0, 0, 0, 0],
    'detune': [-1, 0, -7, -5, 5, -4],
    'transpose': 0,
    'ol': [99, 81, 94, 94, 94, 99],
    'eg_rate': [
		[98, 98, 98, 98, 98, 98],
		[95, 76, 76, 98, 98, 82],
		[46, 58, 49, 87, 87, 87],
		[16, 17, 17, 69, 48, 97]
    ],
    'eg_level': [
		[99, 99, 99, 99, 99, 99],
		[95, 99, 99, 99, 99, 98],
		[95, 99, 99, 99, 99, 99],
		[0, 0, 0, 0, 0, 0]
    ],
    'sensitivity': [1, 3, 0, 2, 2, 0],
}```
<|im_end|>


In [18]:
from utils import parse_last_specs, render_from_specs, validate_specs
import IPython.display as ipd
specs = parse_last_specs(tokenizer.decode(output[0], skip_special_tokens=True))
# validate_specs(specs)
audio = render_from_specs(specs)
ipd.Audio(audio, rate=44100)

ValueError: Failed to parse specs dictionary: closing parenthesis '}' does not match opening parenthesis '[' on line 2 (<unknown>, line 28)

In [14]:
type(specs)

dict